# Pagpapatupad ng Sistema ng Eksperto sa Hayop

Isang halimbawa mula sa [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

Sa halimbawang ito, magpapatupad tayo ng isang simpleng sistema na batay sa kaalaman upang matukoy ang isang hayop batay sa ilang pisikal na katangian. Ang sistema ay maaaring i-representa gamit ang sumusunod na AND-OR tree (ito ay bahagi lamang ng buong puno, madali nating maidaragdag ang iba pang mga patakaran):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.tl.png)


## Ang sarili nating expert systems shell gamit ang backward inference

Subukan nating magtukoy ng isang simpleng wika para sa representasyon ng kaalaman batay sa production rules. Gagamit tayo ng mga klase sa Python bilang mga keyword para tukuyin ang mga patakaran. May tatlong pangunahing uri ng mga klase:
* `Ask` ay kumakatawan sa isang tanong na kailangang itanong sa user. Naglalaman ito ng hanay ng mga posibleng sagot.
* `If` ay kumakatawan sa isang patakaran, at ito ay simpleng syntactic sugar para i-store ang nilalaman ng patakaran.
* `AND`/`OR` ay mga klase para kumatawan sa mga AND/OR na sangay ng puno. Ini-store lang nila ang listahan ng mga argumento sa loob. Upang gawing simple ang code, ang lahat ng functionality ay tinukoy sa parent class na `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Sa aming sistema, ang working memory ay maglalaman ng listahan ng **mga katotohanan** bilang **attribute-value pairs**. Ang knowledgebase ay maaaring tukuyin bilang isang malaking diksyunaryo na nagmamapa ng mga aksyon (mga bagong katotohanan na dapat ipasok sa working memory) sa mga kondisyon, na ipinapahayag bilang AND-OR na mga ekspresyon. Gayundin, ang ilang mga katotohanan ay maaaring `Itanong`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Upang maisagawa ang backward inference, magtatakda tayo ng klase na `Knowledgebase`. Ito ay maglalaman ng:
* Gumaganang `memory` - isang diksyunaryo na nagmamapa ng mga attribute sa mga halaga
* Mga `rules` ng Knowledgebase sa format na itinakda sa itaas

Dalawang pangunahing pamamaraan ay:
* `get` upang makuha ang halaga ng isang attribute, na gumagawa ng inference kung kinakailangan. Halimbawa, ang `get('color')` ay kukunin ang halaga ng isang color slot (magtatanong ito kung kinakailangan, at itatabi ang halaga para sa susunod na paggamit sa working memory). Kung magtanong tayo ng `get('color:blue')`, magtatanong ito para sa kulay, at pagkatapos ay magbabalik ng halaga na `y`/`n` depende sa kulay.
* `eval` ang gumagawa ng aktwal na inference, ibig sabihin, sinusuri ang AND/OR tree, ine-evaluate ang mga sub-goal, at iba pa.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Ngayon, tukuyin natin ang ating kaalaman tungkol sa mga hayop at isagawa ang konsultasyon. Tandaan na ang tawag na ito ay magtatanong sa iyo. Maaari kang sumagot sa pamamagitan ng pag-type ng `y`/`n` para sa mga tanong na oo-hindi, o sa pamamagitan ng pagtukoy ng numero (0..N) para sa mga tanong na may mas mahabang sagot na multiple-choice.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Paggamit ng PyKnow para sa Forward Inference

Sa susunod na halimbawa, susubukan nating ipatupad ang forward inference gamit ang isa sa mga library para sa knowledge representation, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** ay isang library para sa paglikha ng forward inference systems sa Python, na idinisenyo upang maging katulad ng klasikong lumang sistema [CLIPS](http://www.clipsrules.net/index.html).

Maaari rin nating ipatupad ang forward chaining nang tayo mismo nang walang masyadong problema, ngunit ang mga simpleng implementasyon ay kadalasang hindi masyadong epektibo. Para sa mas mahusay na rule matching, isang espesyal na algorithm na tinatawag na [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) ang ginagamit.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Itatakda natin ang ating sistema bilang isang klase na nag-subclass sa `KnowledgeEngine`. Ang bawat panuntunan ay tinutukoy ng isang hiwalay na function na may `@Rule` na anotasyon, na nagsasaad kung kailan dapat mag-trigger ang panuntunan. Sa loob ng panuntunan, maaari tayong magdagdag ng mga bagong katotohanan gamit ang `declare` na function, at ang pagdaragdag ng mga katotohanang iyon ay magreresulta sa pagtawag ng iba pang mga panuntunan ng forward inference engine.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Kapag naitakda na natin ang isang knowledgebase, pinupunan natin ang ating working memory ng ilang paunang impormasyon, at pagkatapos ay tinatawag ang `run()` na pamamaraan upang isagawa ang inference. Makikita mo bilang resulta na ang mga bagong inferred na impormasyon ay idinadagdag sa working memory, kabilang ang panghuling impormasyon tungkol sa hayop (kung tama ang lahat ng paunang impormasyon na itinakda natin).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
